# Load Dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("Kompetenzen_BFI_bereinigt.csv", encoding="utf-8")

# K = Klicker (global pro Modul)
df["Unit"] = df["Unit"].fillna("K")

print(df.head())

In [ ]:
df_multiplied = df.copy()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df['Typ'].value_counts()

In [ ]:
df_multi = df.copy(deep=True)

# Split the column labels at '_' to create MultiIndex columns
split_columns = [col.split('_') for col in df_multi.columns]

# Create a MultiIndex using these split column names
multi_index = pd.MultiIndex.from_tuples(split_columns)

# Assign the new multi-index as columns of the DataFrame
df_multi.columns = multi_index

df_multi = df_multi.set_index([('Modul', None, None, None), ('Unit', None, None, None)])

# rename index levels to just Modul and Unit
df_multi.index.names = ['Modul', 'Unit']

df_multi.head()

In [ ]:
df_multi['Gewicht in Minuten']

# Weighted Competencies

In [ ]:
df_multiplied = df_multi.copy(deep=True)

competency_cols = ["Bloom", "FK", "ÜK", "Fragetyp"]

def get_level_cols(df, level, labels):
    return [col for col in df.columns if col[level] in labels]

# Filter columns where the first level is in competency_cols
selected_columns = get_level_cols(df_multiplied, 0, competency_cols)

# Multiply the selected columns by the 'Gewicht in Minuten' column
df_multiplied[selected_columns] = df_multiplied[selected_columns] * df_multiplied['Gewicht in Minuten'].values

df_multiplied[selected_columns] = df_multiplied[selected_columns].fillna(0)
df_multiplied[selected_columns] = df_multiplied[selected_columns].round(2)

df_multiplied

In [ ]:
xlwriter = pd.ExcelWriter(f"out/results.xlsx", engine='xlsxwriter')

df_multi.round(2).to_excel(xlwriter, sheet_name=f"Original")
df_multiplied.round(2).to_excel(xlwriter, sheet_name=f"Original mal Zeitgewichtung")

for label in ["Bloom", "Fragetyp", "FK", "ÜK"]:
  df_agg_m = df_multiplied[get_level_cols(df_multiplied, 0, [label])].groupby("Modul").sum()
  df_agg_m.round(2).to_excel(xlwriter, sheet_name=f"{label} agg. Modul")

  df_agg_u = df_multiplied[get_level_cols(df_multiplied, 0, [label])].groupby(["Modul", "Unit"]).sum()
  df_agg_u.round(2).to_excel(xlwriter, sheet_name=f"{label}  agg. Modul & Unit")

  df_module_weight_in_min = df_multiplied[get_level_cols(df_multiplied, 0, ["Gewicht in Minuten"])].groupby('Modul').sum()

  df_agg_m_percent = (df_agg_m.div(df_module_weight_in_min['Gewicht in Minuten'].values, axis=1)) * 100
  df_agg_m_percent.round(2).to_excel(xlwriter, sheet_name=f"{label} agg. Modul (%)")

  df_unit_weight_in_min = df_multiplied[get_level_cols(df_multiplied, 0, ["Gewicht in Minuten"])].groupby(['Modul', 'Unit']).sum()

  df_agg_u_percent = (df_agg_u.div(df_unit_weight_in_min['Gewicht in Minuten'].values, axis=1)) * 100
  df_agg_u_percent.round(2).to_excel(xlwriter, sheet_name=f"{label} agg. Modul & Unit (%)")

  df_agg_total = pd.DataFrame(df_agg_m_percent.sum(axis=0))
  df_agg_total = df_agg_total / df_agg_total.sum(axis=0) * 100
  df_agg_total.round(2).to_excel(xlwriter, sheet_name=f"{label} agg. total (%)")

xlwriter.close()
